In [182]:
from bs4 import BeautifulSoup

In [4]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

In [5]:
res = search.by_city_and_state(None, "tx", returns=10000)

In [6]:
len(res)

1745

In [7]:
codes = []
for zip_info in res: 
    codes.append(zip_info.zipcode)

In [9]:
codes[:10]

['75001',
 '75002',
 '75006',
 '75007',
 '75009',
 '75010',
 '75013',
 '75019',
 '75020',
 '75021']

In [10]:
import requests

In [28]:
url = 'https://ziplook.house.gov/htbin/findrep_house?ZIP=75001'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

### single district

In [16]:
import re

In [17]:
def extract_from_zip(zip_code):
    url = 'https://ziplook.house.gov/htbin/findrep_house?ZIP=' + str(zip_code)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    reps = []
    if "Possible Representatives" in soup.text:
        for rep in soup.find_all("div", {"class": "RepInfo"}):
            segs = [zip_code]
            for t in rep.text.split('\t'):
                cleaned = t.strip()
                if len(cleaned) > 0:
                    segs.append(cleaned)
            reps.append(segs)
    else:
        rep = soup.find("div", {'class': re.compile(r"^repdistrict")})
        string = regex.sub('', rep.text)
        m = re.match(r'(.*)in the (.*) of(.*)', string)
        district = m.group(2)
        
        rep = soup.find("div", {"id": "RepInfo"})
        infos = [zip_code]
        for seg in rep.text.split('\t'):
            infos.append(seg.strip())
        infos.append(district)
        reps.append(infos)
    return reps

In [18]:
import time

In [19]:
from tqdm import tqdm

In [36]:
regex = re.compile(r'[\n\r\t]')

In [20]:
database = []

for code in tqdm(codes):
    try:
        database.extend(extract_from_zip(code))
    except:
        pass

100%|██████████| 1745/1745 [09:42<00:00,  3.00it/s]


In [37]:
for code in tqdm(codes):
    if code not in collects:
        try:
            database.extend(extract_from_zip(code))
        except Exception as e: 
            print(e)

  9%|▊         | 151/1745 [00:07<01:04, 24.56it/s]

'NoneType' object has no attribute 'text'


  9%|▉         | 158/1745 [00:08<01:17, 20.46it/s]

'NoneType' object has no attribute 'text'


 48%|████▊     | 844/1745 [00:56<00:27, 33.21it/s]

...

 90%|█████████ | 1573/1745 [01:42<00:17,  9.90it/s]

'NoneType' object has no attribute 'text'


 92%|█████████▏| 1613/1745 [01:46<00:11, 11.28it/s]

'NoneType' object has no attribute 'text'


100%|██████████| 1745/1745 [01:58<00:00, 14.74it/s]


In [38]:
import pandas as pd

In [42]:
data = pd.DataFrame.from_records(database, columns=['zip code', 'rep', 'party', 'district'])

In [44]:
data.to_csv('TX.csv')

In [183]:
members = ['Lloyd Doggett', 'Jodey C. Arrington', 'Kevin Brady', 'Kenny Marchant']

In [190]:
tx_codes = data[data.rep.isin(members)]['zip code'].unique()

In [192]:
len(tx_codes)

312

In [201]:
options = webdriver.ChromeOptions()
options.headless = None

In [202]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, )

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/jiaruixu/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [203]:
driver.set_window_size(3000, 4000)

In [204]:
tx_info = {}

for tx_zip in tqdm(tx_codes):
    reps = []
    url = 'https://wrm.capitol.texas.gov/map?address=&city=&county=&zip='+tx_zip
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # driver.save_screenshot('2.png')
    for card in soup.find_all('div', {'class':'mat-list-text'}):
        rep = card.find('div', {'class': 'label mat-line'}).text
        title = card.find('div', {'class': 'sublabel mat-line'}).text
        reps.append((rep, title))
    tx_info[tx_zip] = reps

100%|██████████| 312/312 [53:05<00:00, 10.21s/it]


In [227]:
def get_numbers(txt):
    return [re.findall(r'\d+', txt)]

In [228]:
get_numbers('U.S. Congressional District 24')

[['24']]

In [229]:
import itertools 
from itertools import product 

code2congress = []
code2legislative = []
code2state_senate = []

for info in tx_info.items():
    zip_code, reps = info
    for rep in reps:
        if 'Congressman' in rep[0] or 'U.S. Representatives' in rep[0]:
            code2congress.append([zip_code] + get_numbers(rep[1]))
        if 'State Representatives' in rep[0]:
            code2legislative.append([zip_code] + get_numbers(rep[1]))
        if 'Representative' in rep[0]:
            code2legislative.append([zip_code] + get_numbers(rep[1]))
        if 'Texas Senate' in rep[1]:
            code2state_senate.append([zip_code] + get_numbers(rep[1]))
        if 'State Senators (' in rep[0]:
            code2state_senate.append([zip_code] + get_numbers(rep[1]))

In [242]:
code2legislative

[['75006', ['65', '102', '103', '115']],
 ['75006', ['65', '102', '103', '115']],
 ['75007', ['24', '26']],
 ['75007', ['65', '115']],
 ['75007', ['65', '115']],
 ['75019', ['24', '26']],
 ['75019', ['65', '115']],
 ['75019', ['65', '115']],
 ['75022', ['24', '26']],
 ['75022', ['63', '98']],
 ['75022', ['63', '98']],
 ['75024', ['3', '24']],
 ['75024', ['33', '66', '106']],
 ['75024', ['33', '66', '106']],
 ['75028', ['24', '26']],
 ['75028', ['63', '65', '98']],
 ['75028', ['63', '65', '98']],
 ['75056', ['24', '26']],
 ['75056', ['65', '106']],
 ['75056', ['65', '106']],
 ['75060', ['103', '104', '105']],
 ['75060', ['103', '104', '105']],
 ['75061', ['24', '33']],
 ['75061', ['103', '105']],
 ['75061', ['103', '105']],
 ['75062', ['24', '33']],
 ['75062', ['92', '103', '105', '115']],
 ['75062', ['92', '103', '105', '115']],
 ['75093', ['3', '24']],
 ['75093', ['65', '66', '106']],
 ['75093', ['65', '66', '106']],
 ['75209', ['24', '30', '32']],
 ['75209', ['103', '108']],
 ['75209

In [235]:
congress_df = pd.DataFrame.from_records(code2congress, columns=['zip', 'congress']).explode('congress')

In [237]:
congress_df.to_csv('tx_congress.csv', index=False)

In [238]:
state_legislative_df = pd.DataFrame.from_records(code2legislative, columns=['zip', 'state_legislative']).explode('state_legislative')

In [239]:
state_legislative_df.to_csv('tx_state_legislative.csv', index=False)

In [240]:
state_senate_df = pd.DataFrame.from_records(code2state_senate, columns=['zip', 'state_senate']).explode('state_senate')

In [241]:
state_senate_df.to_csv('tx_state_senate.csv', index=False)

In [132]:
def get_state_info(zip_code):
    url = 'https://wrm.capitol.texas.gov/map?address=&city=&zip=76427&county='
    print(url)
    driver.set_window_size(3000, 4000)
    driver.get(url)
    driver.implicitly_wait(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # driver.save_screenshot('2.png')
    reps = []
    for card in soup.find_all('mat-card', {'class':'mat-card'}):
        rep = card.find('div', {'class': 'label mat-line'}).text
        title = card.find('div', {'class': 'sublabel mat-line'}).text
        reps.append((rep, title))
    return reps

In [133]:
get_state_info('76427')

https://wrm.capitol.texas.gov/map?address=&city=&zip=76427&county=


[]

In [143]:
url = 'https://wrm.capitol.texas.gov/map?address=&city=&zip=76427&county='
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [144]:
for seg in soup.find_all('mat-card', {'class':'mat-card'}):
    pass

In [145]:
 soup.find_all('mat-card', {'class':'mat-card'})

[]

In [59]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [103]:
options = webdriver.ChromeOptions()
options.headless = True

In [147]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, )

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/jiaruixu/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [149]:
driver.set_window_size(3000, 4000)
url = 'https://wrm.capitol.texas.gov/map?address=&city=&zip=76227&county='
driver.get(url)
driver.implicitly_wait(5)

In [150]:
driver.page_source

'<html lang="en" class="dj_webkit dj_chrome dj_contentbox"><head>\n  <meta charset="utf-8">\n  <title>Who Represents Me?</title>\n  <base href="/">\n\n  <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=0">\n  <meta name="keywords" content="Who Represents Me?">\n  <meta name="author" content="Texas Legislative Council">\n  <meta name="description" content="Who Represents Me provides information about current districts and members of the Texas Senate, Texas House of Representatives, the Texas delegation to the U.S. Senate and House of Representatives, and the State Board of Education.">\n  <link rel="icon" type="image/x-icon" href="favicon3.png">\n  <link href="./assets/roboto/roboto.css" rel="stylesheet">\n  <link href="./assets/materialicons/material-icons.css" rel="stylesheet">\n  <link href="https://cdn.tlc.texas.gov/solar/dvr/arcgis_js_api/library/4.10/esri/css/main.css">\n<link rel="stylesheet" href="styles.d41eea9427b44b1e0fec.css"><style></style><

In [151]:
driver.save_screenshot('3.png')

True

In [82]:

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [141]:
for seg in soup.find_all('mat-card', {'class':'mat-card'}):
    print(seg.text)

In [96]:
seg.contents

[<mat-list-item _ngcontent-c1="" class="mat-list-item mat-2-line"><div class="mat-list-item-content"><div class="mat-list-item-ripple mat-ripple" mat-ripple=""></div><div class="mat-list-text"><div _ngcontent-c1="" class="label mat-line" mat-line="">Senator Ted Cruz</div><div _ngcontent-c1="" class="sublabel mat-line" mat-line="">U.S. Senate</div></div></div></mat-list-item>]

In [95]:
for c in seg.contents:
    print(c)

<mat-list-item _ngcontent-c1="" class="mat-list-item mat-2-line"><div class="mat-list-item-content"><div class="mat-list-item-ripple mat-ripple" mat-ripple=""></div><div class="mat-list-text"><div _ngcontent-c1="" class="label mat-line" mat-line="">Senator Ted Cruz</div><div _ngcontent-c1="" class="sublabel mat-line" mat-line="">U.S. Senate</div></div></div></mat-list-item>


In [97]:
seg.find('div', {'class': 'label mat-line'})

<div _ngcontent-c1="" class="label mat-line" mat-line="">Senator Ted Cruz</div>

In [98]:
seg.find('div', {'class': 'sublabel mat-line'})

<div _ngcontent-c1="" class="sublabel mat-line" mat-line="">U.S. Senate</div>

In [40]:
collects = set()
for row in database:
    collects.add(row[0])

In [41]:
for z in codes:
    if z not in collects:
        print(z)

75245
75258
77097
78241
78262
78470
78471
78473
78474
78475
78476
78477
78478
78798
78799
79320
79405


In [78]:
for rep in soup.find_all("div", {'class': re.compile(r"^repdistrict")}):  
    pass

In [35]:
segs = []
for t in rep.text.split('\t'):
    cleaned = t.strip()
    if len(cleaned) > 0:
        segs.append(cleaned)

In [36]:
for seg in segs: 

['Find Your Representative\n\nInformation provided:\nZip code: 94158',
 'is located in the 12th Congressional district of California.',
 'The representative for this district is:\n\n\n\nNancy Pelosi',
 'Democrat']

In [41]:
rep.text

'\nFind Your Representative\n\nInformation provided:\nZip code: 94158\n\n\n\t\tis located in the 12th Congressional district of California.\n\n\t\tThe representative for this district is:\n\n\n\nNancy Pelosi \n                \tDemocrat\n\n\n\n\n\n'

In [48]:
import re


'Find Your RepresentativeInformation provided:Zip code: 94158is located in the 12th Congressional district of California.The representative for this district is:Nancy Pelosi                 Democrat'

In [23]:
for rep in soup.find_all("div", {"id": "RepInfo"}):
    segs = []
    for t in rep.text.split('\t'):
        cleaned = t.strip()
        if len(cleaned) > 0:
            segs.append(cleaned)

In [51]:
import re
string = regex.sub('', rep.text)
m = re.match(r'(.*)in (.*) of(.*)', string)
print( m.group(2))

the 12th Congressional district


In [65]:
t = '\n\nNancy Pelosi \n                \tDemocrat\n\n\n\n'

In [68]:
infos = []
for seg in t.split('\t'):
    infos.append(seg.strip())

In [79]:
rep.text

'\nFind Your Representative\n\nInformation provided:\nZip code: 94158\n\n\n\t\tis located in the 12th Congressional district of California.\n\n\t\tThe representative for this district is:\n\n\n\nNancy Pelosi \n                \tDemocrat\n\n\n\n\n\n'